# Pre Processing Step
We want to process the data for LDA like:
- Keep NOUNS only as they are the elements that mostly hold aspect value
- Split reviews in sentences (Avoid topic modelling in a too broad sense) (https://aclanthology.org/N10-1122.pdf)

In [ ]:
from gensim import corpora
import pandas as pd

# LDA does not benefit from repeated words among many documents. So we should clean them as they are stopwords.
corpus_path = "../dataset/output/default_sentences/pre_processed.310k.csv"
ds = pd.read_csv(corpus_path)['comments'].apply(lambda x: x.split(' '))

dictionary = corpora.Dictionary(ds)
doc_frequency = {}

for index in range(len(dictionary)):
    doc_frequency[dictionary.get(index)] = len(ds[[dictionary.get(index) in x for x in ds]])
pd.DataFrame.from_dict(doc_frequency, orient='index')[0].map(lambda x: x / len(ds)).sort_values(ascending=False)

Are 'game'(40%), 'play'(20%) and '\<game_name\>'(11%) stopwords? <br>
For sure 'game' is! What about the other two? I believe they bring no context.
We remove them!

In [1]:
# Common default imports
from pprint import pprint
from model_manager import LDAManager
from main.lda.config import LdaGeneratorConfig
from gensim import corpora
import pandas as pd
from gensim.corpora import Dictionary

# Stop words identified
stop_words = ['game', 'play', '<game_name>']

### Default: Sentences slit

In [ ]:
corpus_path = "../dataset/output/default_sentences/pre_processed.310k.csv"
test_corpus_path = "../dataset/output/default_sentences/pre_processed.310k.test.csv"

model_manager = LDAManager.from_scratch(LdaGeneratorConfig(name="default_sentences"), stop_words)
model = model_manager.get_model(corpus_path, False, True)
res = model_manager.evaluate(test_corpus_path)

pprint(res)

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from gensim.corpora import Dictionary
import pandas as pd

dictionary: Dictionary = model.id2word
# Load texts to
texts = pd.read_csv(test_corpus_path)['comments'].apply(lambda x: x.split(' '))

# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(model, texts.apply(lambda x: dictionary.doc2bow(x)).tolist(), dictionary)
vis

### POS: Nouns only

In [ ]:
from pre_processing import extract_pos_ds
import pandas as pd

print("Creating the __noun filtered ds:")
corpus_path = "../dataset/output/pos_tagged_sentence_level/pre_processed.310k.csv"
store_path = "../dataset/output/pos_tagged_sentence_level/pre_processed.310k.noun_only.csv"
extract_pos_ds(pd.read_csv(corpus_path)['comments'], "__noun", store_path)
print("ds created under: " + store_path)

print("Creating the __noun filtered test ds:")
corpus_path = "../dataset/output/pos_tagged_sentence_level/pre_processed.310k.test.csv"
store_path = "../dataset/output/pos_tagged_sentence_level/pre_processed.310k.noun_only.test.csv"
extract_pos_ds(pd.read_csv(corpus_path)['comments'], "__noun", store_path)
print("ds created under: " + store_path)

Now we check if the NOUN only approach works better. <br>
Take nouns only of the POS tagged ds:

In [ ]:
corpus_path = "../dataset/output/pos_tagged_sentence_level/pre_processed.310k.noun_only.csv"
test_corpus_path = "../dataset/output/pos_tagged_sentence_level/pre_processed.310k.noun_only.test.csv"

model_manager = LDAManager.from_scratch(LdaGeneratorConfig(name="default_sentences"), stop_words)
model = model_manager.get_model(corpus_path, False, True)
res = model_manager.evaluate(test_corpus_path)

pprint(res)

In [14]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from gensim.corpora import Dictionary
import pandas as pd

dictionary: Dictionary = model.id2word
# Load texts to
texts = pd.read_csv(test_corpus_path)['comments'].apply(lambda x: x.split(' '))

# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(model, texts.apply(lambda x: dictionary.doc2bow(x)).tolist(), dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
13    -0.233209 -0.005561       1        1  9.198038
12    -0.101986  0.047204       2        1  8.843128
7      0.202171 -0.143089       3        1  8.535858
2      0.080518  0.066422       4        1  8.031112
1     -0.013881  0.184347       5        1  7.622198
8     -0.160835 -0.157964       6        1  7.480469
9      0.038786  0.176612       7        1  7.444389
10    -0.167064 -0.120826       8        1  7.349323
6      0.072377 -0.157634       9        1  6.967530
11     0.065899  0.096497      10        1  6.704844
0     -0.078668  0.072741      11        1  5.783340
5      0.202150 -0.065068      12        1  5.691101
4      0.053988  0.062474      13        1  5.253688
3      0.039752 -0.056154      14        1  5.094982, topic_info=       Term         Freq        Total Category  logprob  loglift
174    card  7263.000000  7263.000000  Default  30.0000  30.0000
147    good  6259.000000  6259.000000  Default  29.0000  29.0000
142    well  2449.000000  2449.000000  Default  28.0000  28.0000
133    time  4740.000000  4740.000000  Default  27.0000  27.0000
209    love  2398.000000  2398.000000  Default  26.0000  26.0000
..      ...          ...          ...      ...      ...      ...
146    come   129.759035  1385.767935  Topic14  -5.3402   0.6086
128    make   133.108429  1822.262821  Topic14  -5.3147   0.3602
24   design   116.141080  1174.316738  Topic14  -5.4511   0.6633
147    good   117.176215  6259.676425  Topic14  -5.4422  -1.0013
158    need   111.652030  2225.565038  Topic14  -5.4905  -0.0155

[1080 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
694       1  0.055129  <number>
694       2  0.017541  <number>
694       4  0.001253  <number>
694       5  0.717934  <number>
694       6  0.005012  <number>
...     ...       ...       ...
207      11  0.019068      year
207      13  0.937261      year
739      10  0.952437         z
8626     10  0.984800       zee
8117     10  0.979445       zen

[4552 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 13, 8, 3, 2, 9, 10, 11, 7, 12, 1, 6, 5, 4])

In [ ]:
from pre_processing import extract_pos_ds
import pandas as pd

print("Creating the __noun filtered ds:")
corpus_path = "../dataset/output/pos_tagged/pre_processed.310k.csv"
store_path = "../dataset/output/pos_tagged/pre_processed.310k.noun_only.csv"
extract_pos_ds(pd.read_csv(corpus_path)['comments'], "__noun", store_path)
print("ds created under: " + store_path)

print("Creating the __noun filtered test ds:")
corpus_path = "../dataset/output/pos_tagged/pre_processed.310k.test.csv"
store_path = "../dataset/output/pos_tagged/pre_processed.310k.noun_only.test.csv"
extract_pos_ds(pd.read_csv(corpus_path)['comments'], "__noun", store_path)
print("ds created under: " + store_path)

In [ ]:
corpus_path = "../dataset/output/pos_tagged/pre_processed.310k.noun_only.csv"
test_corpus_path = "../dataset/output/pos_tagged/pre_processed.310k.noun_only.test.csv"

model_manager = LDAManager.from_scratch(LdaGeneratorConfig(name="default_sentences"), stop_words)
model = model_manager.get_model(corpus_path, False, True)
res = model_manager.evaluate(test_corpus_path)

pprint(res)

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from gensim.corpora import Dictionary
import pandas as pd

dictionary: Dictionary = model.id2word
# Load texts to
texts = pd.read_csv(test_corpus_path)['comments'].apply(lambda x: x.split(' '))

# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(model, texts.apply(lambda x: dictionary.doc2bow(x)).tolist(), dictionary)
vis

Following the experiments I decided to follow the path of nouns only composition document. <br>
The hyperparameters tuning is now the next step.

Noun only on sentences model performs way worse, let's see if going for non LocalLDA but focusing on NOUNS only is viable:

## Tuned best found K values
Hyperparameters tuning yielded on average the following best topics:
- Default sentences: [10]
- NOUN only full review: [7, 12]

### Default sentences:

In [ ]:
corpus_path = "../dataset/output/default_sentences/pre_processed.310k.csv"
test_corpus_path = "../dataset/output/default_sentences/pre_processed.310k.test.csv"

model_manager = LDAManager.from_scratch(
    LdaGeneratorConfig(name="sentences_k_10", topics=10, alpha=.1, eta=.01), stop_words
)
model = model_manager.get_model(corpus_path, False, True)
res = model_manager.evaluate(test_corpus_path)

pprint(res)

Visualize the result:

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from gensim.corpora import Dictionary
import pandas as pd

dictionary: Dictionary = model.id2word
# Load texts to
texts = pd.read_csv(test_corpus_path)['comments'].apply(lambda x: x.split(' '))

# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(model, texts.apply(lambda x: dictionary.doc2bow(x)).tolist(), dictionary)
vis

## NOUNs only

In [ ]:
 # Default approach on the sentences only to avoid global topic recognition
from model import LdaModelGenerator
from main.lda.config import LdaGeneratorConfig

stop_words = ['game', 'play', '<game_name>']

corpus_path = "../dataset/output/pos_tagged/pre_processed.310k.noun_only.csv"
config = LdaGeneratorConfig(name="sentences_k_10", topics=12)
lda, dictionary = LdaModelGenerator(config, stop_words).make_model(corpus_path)

In [ ]:
corpus_path = "../dataset/output/default_sentences/pre_processed.310k.csv"
test_corpus_path = "../dataset/output/default_sentences/pre_processed.310k.test.csv"

model_manager = LDAManager.from_scratch(
    LdaGeneratorConfig(name="sentences_k_10", topics=12, alpha=.1, eta=.01), stop_words
)
model = model_manager.get_model(corpus_path, False, True)
res = model_manager.evaluate(test_corpus_path)

pprint(res)

In [ ]:
from gensim.models import CoherenceModel
import pandas as pd

test_ds = pd.read_csv("../dataset/output/pos_tagged/pre_processed.310k.noun_only.test.csv")

# For the c_v model
texts = test_ds['comments'].apply(lambda x: x.split(' '))

perplexity = lda.log_perplexity(texts.apply(lambda x: dictionary.doc2bow(x)).tolist())
results = dict(cv_coh=[], npmi_coh=[], topn=[3, 10, 25], perplexity=perplexity)

for topn in results['topn']:
    cv_coh = CoherenceModel(lda, texts=texts, coherence='c_v', topn=topn)
    npmi_coh = CoherenceModel(lda, texts=texts, coherence='c_npmi', topn=topn)
    results['cv_coh'].append(cv_coh.get_coherence())
    results['npmi_coh'].append(npmi_coh.get_coherence())

In [ ]:
print(
    f"Model perplexity: {results['perplexity']}\n"
    f"With topn = {results['topn']} we have: \n"
    f" - CV coherence: {results['cv_coh']}\n"
    f" - NPMI coherence: {results['npmi_coh']}\n"
)

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda, texts.apply(lambda x: dictionary.doc2bow(x)).tolist(), dictionary)
vis

In [4]:
# Run for K = 7
corpus_path = "../dataset/output/pos_tagged/pre_processed.310k.noun_only.csv"
test_ds_path = "../dataset/output/pos_tagged/pre_processed.310k.noun_only.test.csv"
res, lda, dictionary = run_procedure(corpus_path, test_ds_path, LdaGeneratorConfig(name="sentences_k_7", topics=12))

pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda, texts.apply(lambda x: dictionary.doc2bow(x)).tolist(), dictionary)
vis

Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/195789 [00:00<?, ?it/s]

Model perplexity: -8.208558033118516
With topn = [3, 10, 25] we have: 
 - CV coherence: [0.7180530229398828, 0.6123247263225092, 0.5880882077576267]
 - NPMI coherence: [0.05778951271585514, 0.023637662956389766, 0.005265168985537204]



NameError: name 'texts' is not defined